In [3]:
# 数据读入
import matplotlib as mlp
import pandas as pd
import numpy as np
dataset = 'gsdtsr'
df = pd.read_csv('./' + dataset + '.csv', sep=',', parse_dates=['LastRun'])
print(df)

            Id  Name  Duration  CalcPrio             LastRun  \
0       548888  1400       450         0 2016-01-10 03:37:20   
1       195761   457     95430         0 2016-01-04 19:33:07   
2       250961   559      6088         0 2016-01-13 15:49:38   
3       685761  2415       330         0 2016-01-15 16:57:02   
4      1058480  4496     26751         0 2016-01-06 01:07:20   
...        ...   ...       ...       ...                 ...   
12601   627216  2013       344         0 2016-01-09 20:25:53   
12602   875511  3450      1052         0 2016-01-09 15:23:22   
12603   758755  2748     34121         0 2016-01-05 01:03:41   
12604  1165475  4800      4290         0 2016-01-16 11:09:21   
12605   601092  1874        95         0 2016-01-06 01:05:07   

                                             LastResults  Verdict  Cycle  
0      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...        0    177  
1      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...        0     68  
2     

In [4]:
# Genarate ZeroCount. 统计倒数前n个 默认n=4
df['LastResults'] = df['LastResults'].astype(str)
n = 8
# dfZeCo = df[df['LastResults'].str.len() > 9]
def f(x):
    x = x[::-1]
    count = 0
    for i in range(min(len(x), 3 * n - 1)):
      if x[i] == '0':
        count += 1
    return count
df['ZeroCount'] = df['LastResults'].apply(f)
df['ZeroCountNormal'] = df['ZeroCount'].apply(lambda x : x / n)
print(df)
# df.to_csv('./'+ dataset + '_ZeroCount_' + 4 + '.csv', index=False)

            Id  Name  Duration  CalcPrio             LastRun  \
0       548888  1400       450         0 2016-01-10 03:37:20   
1       195761   457     95430         0 2016-01-04 19:33:07   
2       250961   559      6088         0 2016-01-13 15:49:38   
3       685761  2415       330         0 2016-01-15 16:57:02   
4      1058480  4496     26751         0 2016-01-06 01:07:20   
...        ...   ...       ...       ...                 ...   
12601   627216  2013       344         0 2016-01-09 20:25:53   
12602   875511  3450      1052         0 2016-01-09 15:23:22   
12603   758755  2748     34121         0 2016-01-05 01:03:41   
12604  1165475  4800      4290         0 2016-01-16 11:09:21   
12605   601092  1874        95         0 2016-01-06 01:05:07   

                                             LastResults  Verdict  Cycle  \
0      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...        0    177   
1      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...        0     68   
2  

In [8]:
# Genarate IdleTime. 生成闲置时间，并删除无用数据
lines = df.shape[0]
df['IdleTime'] = [None] * lines
dict = {}
# key:df.loc[i, 'Name'] value:df.loc[i, 'LastRun']
for i in range(0, lines):
    key = df.loc[i, 'Name']
    value = df.loc[i, 'LastRun']
    if key in dict:
        df.loc[i, 'IdleTime'] = df.loc[i, 'LastRun'] - dict[key]
    dict[key] = value
print(df.loc[0 : lines])

          Id    Name  Duration             LastRun  \
0          1   78082     57262 2015-02-13 16:13:00   
1          2   78083     57145 2015-02-13 16:13:00   
2          3   78084     56861 2015-02-13 16:13:00   
3          4   78085     55332 2015-02-13 16:13:00   
4          5   78086     59120 2015-02-13 16:13:00   
...      ...     ...       ...                 ...   
32255  32256   82981     57744 2016-10-25 16:18:00   
32256  32257  102163     55844 2016-10-25 16:20:00   
32257  32258  102162     55237 2016-10-25 16:20:00   
32258  32259  124422     61682 2016-10-25 16:20:00   
32259  32260  124086     62265 2016-10-25 16:20:00   

                                             LastResults  Verdict  Cycle  \
0                                                     []        1      1   
1                                                     []        1      1   
2                                                     []        1      1   
3                                              

In [9]:
# IdleTime 标准化
import datetime
from datetime import timedelta
max = timedelta(days=0)
for i in range(0, lines):
  if df.loc[i, 'IdleTime'] != None and max < df.loc[i, 'IdleTime']:
    max = df.loc[i, 'IdleTime']
print(max)
df['IdleTimeNormal'] = [None] * lines
def r(x, y = max):
  if pd.isnull(x):
    return 0
  return x / y;
df['IdleTimeNormal'] = df['IdleTime'].apply(r)
# for i in range(0, 32260):
#   if df.loc[i, 'IdleTimeNormal'] == None:
#     df.loc[i, 'IdleTimeNormal'] = 0

# 剩余标准化
df[['DurationNormal', 'CycleNormal']] = df[['Duration', 'Cycle']].apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
print(df.loc[0 : lines])

# 删除无用数据
df = df[['DurationNormal', 'CycleNormal', 'IdleTimeNormal', 'ZeroCountNormal', 'Verdict']]
df = df.round(5)
print(df.loc[0 : lines])

# 输出
df.to_csv('./' + dataset + '_normal.csv', index=False, sep=' ')

203 days 19:56:00
          Id    Name  Duration             LastRun  \
0          1   78082     57262 2015-02-13 16:13:00   
1          2   78083     57145 2015-02-13 16:13:00   
2          3   78084     56861 2015-02-13 16:13:00   
3          4   78085     55332 2015-02-13 16:13:00   
4          5   78086     59120 2015-02-13 16:13:00   
...      ...     ...       ...                 ...   
32255  32256   82981     57744 2016-10-25 16:18:00   
32256  32257  102163     55844 2016-10-25 16:20:00   
32257  32258  102162     55237 2016-10-25 16:20:00   
32258  32259  124422     61682 2016-10-25 16:20:00   
32259  32260  124086     62265 2016-10-25 16:20:00   

                                             LastResults  Verdict  Cycle  \
0                                                     []        1      1   
1                                                     []        1      1   
2                                                     []        1      1   
3                            

In [5]:
# 临时缩放
import matplotlib as mlp
import pandas as pd
import numpy as np
dataset = 'rails'
df = pd.read_csv('./' + dataset + '_normal.csv', sep=' ')
df['IdleTimeNormal'] = df['IdleTimeNormal'].apply(lambda x: x * 10)
df['DurationNormal'] = df['DurationNormal'].apply(lambda x: x * 10)
df.to_csv('./' + dataset + '_normal.csv', index=False, sep=' ')
